In [1]:
import requests
import time
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy
from dotenv import load_dotenv
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

load_dotenv()

True

### Verbindung zur Datenbank aufbauen und Daten aus der Tabelle laden

In [2]:
host = os.getenv('HOST')
port = os.getenv('PORT')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
database= os.getenv('DATABASE')

In [3]:
# Verbindung zur Datenbank herstellen
connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
print("Verbindung zur Datenbank erfolgreich hergestellt.")

# Einen Cursor erstellen, um SQL-Abfragen auszuführen
cursor = connection.cursor()

# SQL SELECT-Abfrage
select_query = "SELECT * FROM value;"

# Abfrage ausführen
cursor.execute(select_query)

# Ergebnis abrufen
rows = cursor.fetchall()

# Ergebnis in ein Pandas DataFrame umwandeln
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Cursor und Verbindung schließen
cursor.close()
connection.close()


Verbindung zur Datenbank erfolgreich hergestellt.


In [4]:
df_new = df

### Random Forest Modell

In [5]:
# 1. Convert the time column to datetime objects
df['time'] = pd.to_datetime(df['time'])

# 2. Feature Engineering with Lags
lags = [1, 2, 3]  # Lags for 10, 20, and 30 minutes
for lag in lags:
    df[f'Temperatur_lag_{lag*10}'] = df.groupby('senseid_fk')['temperature'].shift(lag)

# 3. Create target variables for each prediction horizon
for minutes in [10, 20, 30]:
    shift = minutes // 10  # Calculate the shift based on 10-minute intervals
    df[f'Temperatur_in_{minutes}_Minuten'] = df.groupby('senseid_fk')['temperature'].shift(-shift)

# 4. Remove rows with missing values (introduced by lagging and shifting)
df.dropna(inplace=True)

In [6]:
# 5. Define target variables and create a dictionary for models
target_variables = ['Temperatur_in_10_Minuten', 'Temperatur_in_20_Minuten', 'Temperatur_in_30_Minuten']
models = {}

# 6. Train and evaluate models for each target variable
for target in target_variables:
    # 6.1. Select features and target variable
    X = df.drop(target_variables + ["time","index"], axis=1)
    y = df[target]
    
    # 6.2. Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 6.3. Define the preprocessing for the 'senseid_fk' column
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), ['senseid_fk'])
        ],
        remainder='passthrough'
    )
    
    # 6.4. Create the pipeline with preprocessing and model training
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(n_estimators=350, max_depth=55, random_state=42, n_jobs=-1))
    ])
    
    # 6.5. Train the model
    pipeline.fit(X_train, y_train)
    
    # 6.6. Make predictions and evaluate
    y_pred = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"\n{target}:")
    print(f"  Mean Squared Error: {mse}")
    print(f"  Mean Absolute Error: {mae}")
    print(f"  R^2 Score: {r2}")
    
    # 6.7. Store the trained model
    models[target] = pipeline


Temperatur_in_10_Minuten:
  Mean Squared Error: 0.13823168291792626
  Mean Absolute Error: 0.11765488398884175
  R^2 Score: 0.998341611408165

Temperatur_in_20_Minuten:
  Mean Squared Error: 0.27475684100421754
  Mean Absolute Error: 0.18680818969057736
  R^2 Score: 0.9967049032039207

Temperatur_in_30_Minuten:
  Mean Squared Error: 0.4070045273572283
  Mean Absolute Error: 0.2464608002963044
  R^2 Score: 0.9951192871241138


### predictions

In [11]:
# Verbindung zur Datenbank herstellen
connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
print("Verbindung zur Datenbank erfolgreich hergestellt.")

# Einen Cursor erstellen, um SQL-Abfragen auszuführen
cursor = connection.cursor()

# SQL SELECT-Abfrage
select_query = """
    SELECT *
    FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY senseid_fk ORDER BY time DESC) AS r
        FROM value
    ) sub
    WHERE r <= 4;
"""

# Abfrage ausführen
cursor.execute(select_query)

# Ergebnis abrufen
rows = cursor.fetchall()

# Ergebnis in ein Pandas DataFrame umwandeln
df_new = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Cursor und Verbindung schließen
cursor.close()
connection.close()


Verbindung zur Datenbank erfolgreich hergestellt.


In [12]:
df_new.head()

,index,time,temperature,luftfeuchtigkeit,pm10,pm2_5,senseid_fk,r
0,1715125,2024-06-08 13:40:09,26.9,80.2,8.30,7.47,590e0b0a51d3460011c725c4,1
1,1715117,2024-06-08 13:30:07,28.0,80.9,6.87,6.20,590e0b0a51d3460011c725c4,2
2,1715109,2024-06-08 13:20:08,26.8,81.5,7.23,6.50,590e0b0a51d3460011c725c4,3
3,1715101,2024-06-08 13:10:08,29.0,79.8,8.43,7.03,590e0b0a51d3460011c725c4,4
4,1715120,2024-06-08 13:40:07,26.3,54.6,4.93,4.30,5ae4a726223bd80019a367a6,1


In [13]:
# Zeitspalte in datetime-Objekte umwandeln
df_new['time'] = pd.to_datetime(df_new['time'])

# Berechne die Lags für die neuen Daten
lags = [1, 2, 3]  # Lags für 10, 20 und 30 Minuten
for lag in lags:
    df_new[f'Temperatur_lag_{lag*10}'] = df_new.groupby('senseid_fk')['temperature'].shift(lag)

# Fehlende Werte durch geeignete Werte ersetzen, falls notwendig
#df_all.fillna(method='ffill', inplace=True)

# Liste der einzigartigen Stationen
stations = df_new['senseid_fk'].unique()

# Dictionary zum Speichern der Vorhersagen für jede Station
all_predictions = {}

for station in stations:
    # Daten für die aktuelle Station auswählen
    df_station = df_new[df_new['senseid_fk'] == station]
    
    # Neueste Zeile für Vorhersagen auswählen
    latest_row = df_station.iloc[[-1]]
    
    # Vorhersagen treffen
    predictions = {}
    for target in target_variables:
        model = models[target]
        # Die Features für das Modell auswählen (identisch zu den Trainingsdaten)
        X_new = latest_row.drop(target_variables + ["time", "index"], axis=1, errors='ignore')
        
        # Vorhersage durchführen
        prediction = model.predict(X_new)
        predictions[target] = prediction[0]
    
    # Ergebnisse für die aktuelle Station speichern
    all_predictions[station] = predictions

# Ergebnisse in ein DataFrame umwandeln
df_predictions = pd.DataFrame(all_predictions)

In [14]:
# Ergebnisse anzeigen
df_predictions.rename(columns={"5ae4a726223bd80019a367a6":"Mersch",
                      "5dde9523ba7944001da4150e":"Ettlingen",
                      "5c08379b1c28f9001a3f580c":"Stuttgart",
                      "5b4d11485dc1ec001b5452c7":"München",
                      "65e8d93acbf5700007f920ca":"Berlin",
                      "590e0b0a51d3460011c725c4":"Hamburg",
                      "605f498077a88b001bba3dc0":"Dortmund",
                      "64722d1c9be0580007f776d9":"Jena"}, inplace=True)

df_predictions

,Hamburg,Mersch,München,Stuttgart,Ettlingen,Dortmund,Jena,Berlin
Temperatur_in_10_Minuten,30.246343,25.890571,19.545657,34.094800,29.702429,22.759875,28.109086,29.311629
Temperatur_in_20_Minuten,29.544457,25.788857,19.181171,34.331914,29.547600,22.740178,28.013200,29.247029
Temperatur_in_30_Minuten,29.419486,25.804400,19.327029,34.471800,29.494571,22.729926,28.287429,29.231457
